# Imports
Import python libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import random

from tqdm import tqdm
from itertools import combinations, product

# Input and output directories
Where the sql database is and which folder the results are saved.

In [42]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')

# path to images
image_dir = os.path.join(home, 'video_resources','image_datasets')

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'experiment')

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/jmacarulla/anaconda3/envs/lr-video/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/jmacarulla/anaconda3/envs/lr-video/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

# Images to show

In [ ]:
dif1 ='xqlfw/Donatella_Versace/Donatella_Versace_0003.jpg'
dif2 = 'xqlfw/Mona_Ayoub/Mona_Ayoub_0001.jpg'

same1='xqlfw/George_W_Bush/George_W_Bush_0223.jpg'
same2='xqlfw/George_W_Bush/George_W_Bush_0183.jpg'
	


# Get images and datafram with same identity pairs

In [ ]:
imagenes = pd.read_pickle(os.path.join(output_dir,'imagenes.pkl'))
pd_misma = pd.read_pickle(os.path.join(output_dir,'misma_identidad.pkl'))
misma_aleatorio = pd_misma.sample(frac=1).reset_index(drop=True)
pd_dif = pd.DataFrame(columns = pd_misma.columns)

In [ ]:
n = len(imagenes)
m = len(pd_misma)
ntotal = int(n*(n-1)/2)

aleatoria = random.sample(range(ntotal),2*m)

# generador de diferentes

In [ ]:
diferentes = ((a[1],b[1]) for a,b in combinations(imagenes.iterrows(), 2) if a[1].identity != b[1].identity)

Defino la funcion de distancia o similitud

In [ ]:
#hago la funcion coseno por hacer una
def similitud(a,b):
    a1 = np.array(a)
    b1 = np.array(b)

    return( np.dot(a1,b1)/(np.linalg.norm(a1)*np.linalg.norm(b1)))


obtener diferentes

In [ ]:
n = len(aleatoria)
for indice in range (ntotal):
    dif = next(diferentes)
    if indice in aleatoria:
        pd_dif = pd_dif.append({
            'path1': dif[0].path,
            'path2': dif[1].path,
            'resultado': similitud(dif[0].embeddings,dif[1].embeddings)
            }, ignore_index=True)
        n-=1
        print(n)
    if not n:
        break            
        

In [ ]:
pd_dif.to_pickle(os.path.join(output_dir,'diferentes.pkl'))

In [ ]:
#same = [(a[1],b[1]) for a,b in combinaciones if a[1].identity == b[1].identity]

In [ ]:
same = []
different = []

In [ ]:
for a,b in tqdm(combinaciones):
    if a[1].identity == b[1].identity:
        same.append([a[1].image_id,b[1].image_id])
    else:
        different.append([a[1].image_id,b[1].image_id])

In [ ]:
df_resultado = pd.DataFrame(columns=['path1', 'path2', 'resultado'])

In [ ]:
#identities = set(df1.identity)
identities = []

In [ ]:

for identity in tqdm(identities) :
    dfx = df1.loc[(df1.identity == identity)]
    if len(dfx) > 1:
        combinaciones = list(combinations(dfx.iterrows(), 2))

        # Iterar sobre las combinaciones
        for comb in tqdm(combinaciones):
            index1, row1 = comb[0]
            index2, row2 = comb[1]
    
            path1 = row1['path']
            path2 = row2['path']
    
            # Realizar la operación deseada (ejemplo: suma de los embeddings)
            resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
            df_resultado = df_resultado.append({
            'path1': path1,
            'path2': path2,
            'resultado': resultado
            }, ignore_index=True)
        
        


In [ ]:
if len(df_resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'misma_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'misma_identidad.xlsx'))

Buscar en diferentes identidades

In [ ]:
identities = list(set(df1.identity))
#identities = []

In [ ]:
df_resultado = pd.DataFrame(columns=['path1', 'path2', 'resultado'])

In [ ]:
for i,identity in enumerate(identities):
    print(i)
    dfx = df1.loc[(df1.identity == identity)]
    dfy = df1.loc[(df1.identity > identity)]
    
    combinaciones = list(product(dfx.iterrows(), dfy.iterrows()))

        # Iterar sobre las combinaciones
    df_parcial = pd.DataFrame(columns=['path1', 'path2', 'resultado'])
    for comb in tqdm(combinaciones):
        
        index1, row1 = comb[0]
        index2, row2 = comb[1]
    
        path1 = row1['path']
        path2 = row2['path']
    
        # Realizar la operación deseada (ejemplo: suma de los embeddings)
        resultado = similitud(row1['embeddings'] , row2['embeddings'])
    
            # Agregar los resultados al DataFrame de resultado
        df_parcial = df_parcial.append({
        'path1': path1,
        'path2': path2,
        'resultado': resultado
        }, ignore_index=True)

        if len(df_parcial)>1000:
            df_ordenado = df_parcial.sort_values(by='resultado', ascending=False)
            df_parcial = df_ordenado.head(10)

    #para cada identidad solo cogemos las 10 imágenes que más se le parecen     
    df_ordenado = df_parcial.sort_values(by='resultado', ascending=False)
    df_resultado = df_resultado.append(df_ordenado.head(10), ignore_index=True)
    df_resultado.to_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))
        

In [ ]:
df_resultado.to_excel(os.path.join(output_dir,'diferente_identidad_0.xlsx'))

In [ ]:
df_resultado = pd.read_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))

In [ ]:
if len(df_resultado):
    df_resultado.to_pickle(os.path.join(output_dir,'diferente_identidad.pkl'))
    df_resultado.to_excel(os.path.join(output_dir,'diferente_identidad.xlsx'))

In [ ]:
len(identities)